##### Copyright 2022 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: retrieval

This tutorial is a slightly adapted version of the [basic retrieval tutorial](https://www.tensorflow.org/recommenders/examples/basic_retrieval#fitting_and_evaluating) from TensorFlow Recommenders documentation.

## Imports


Let's first get our imports out of the way.

In [2]:
import datetime
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

c:\Users\LinDNan\.conda\envs\app\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preparing the dataset

In [3]:
# Ratings data.
df = pd.read_csv("./users.csv", sep=",", encoding='utf-8')
ratings = tf.data.Dataset.from_tensor_slices(dict(df))
# Features of all the available movies.
df = pd.read_csv("./stores.csv", sep=",", encoding='utf-8')
stores = tf.data.Dataset.from_tensor_slices(dict(df))
# Keep only 'movie_title' and 'user_id'
ratings = ratings.map(lambda x: {
    "store": x["store_name"],
    "user_id": x["id"],
})
stores = stores.map(lambda x: x["name"])

Let's use a random split, putting 80% of the ratings in the train set, and 20% in the test set.

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(10_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(8_000)
test = shuffled.skip(8_000).take(2_000)

Next we figure out unique user ids and movie titles present in the data so that we can create the embedding user and movie embedding tables.

In [5]:
movie_titles = stores.batch(1_000)
user_ids = ratings.batch(10_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Implementing a retrieval model

We are going to building a two-tower retrieval model by building each tower separately and then combining them in the final model.

### The query tower

The first step is to decide on the dimensionality of the query and candidate representations:

In [6]:
embedding_dimension = 32


The second is to define the model itself. 

In [7]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### The candidate tower

We can do the same with the candidate tower.

In [8]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### Metrics

We use the `tfrs.metrics.FactorizedTopK` metric for our retrieval model.

In [9]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=stores.batch(128).map(movie_model)
)

### Loss

The next component is the loss used to train our model. We'll make use of the `Retrieval` task object: a convenience wrapper that bundles together the loss function and metric computation:

In [10]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

### The full model

We can now put it all together into a model.

In [11]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["store"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

## Fitting and evaluating

Let's instantiate the model now.

In [12]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [13]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [14]:
model.fit(cached_train, epochs=3)

Epoch 1/3
1/1 [==============================] - 2s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0019 - factorized_top_k/top_100_categorical_accuracy: 0.0044 - loss: 71898.0078 - regularization_loss: 0.0000e+00 - total_loss: 71898.0078
Epoch 2/3
1/1 [==============================] - 1s 852ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0025 - factorized_top_k/top_50_categorical_accuracy: 0.0393 - factorized_top_k/top_100_categorical_accuracy: 0.1105 - loss: 71772.1562 - regularization_loss: 0.0000e+00 - total_loss: 71772.1562
Epoch 3/3
1/1 [==============================] - 1s 850ms/step - factorized_top_k/top_1_categorical_accuracy: 1.2500e-04 - factorized_top_k/top_5_categor

Finally, we can evaluate our model on the test set:

In [15]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 392ms/step - factorized_top_k/top_1_categorical_accuracy: 5.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 5.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0010 - factorized_top_k/top_50_categorical_accuracy: 0.0065 - factorized_top_k/top_100_categorical_accuracy: 0.0110 - loss: 15207.5049 - regularization_loss: 0.0000e+00 - total_loss: 15207.5049


{'factorized_top_k/top_1_categorical_accuracy': 0.0005000000237487257,
 'factorized_top_k/top_5_categorical_accuracy': 0.0005000000237487257,
 'factorized_top_k/top_10_categorical_accuracy': 0.0010000000474974513,
 'factorized_top_k/top_50_categorical_accuracy': 0.006500000134110451,
 'factorized_top_k/top_100_categorical_accuracy': 0.010999999940395355,
 'loss': 15207.5048828125,
 'regularization_loss': 0,
 'total_loss': 15207.5048828125}

## Making predictions

Now that we have a model, we would like to be able to make predictions. We can use the `tfrs.layers.factorized_top_k.BruteForce` layer to do this.

In [16]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((stores.batch(100), stores.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'\xe5\xa1\x91\xe5\x8f\xb0\xe7\x9f\xb3\xe5\x8c\x96\xe6\x9c\x9d\xe8\x8c\xb6\xe6\x9a\xae\xe9\x85\x92 Bar - MinLife Store'
 b'\xe5\xa1\x91\xe5\x8f\xb0\xe7\x9f\xb3\xe5\x8c\x96\xe6\x9c\x9d\xe8\x8c\xb6\xe6\x9a\xae\xe9\x85\x92 Bar - MinLife Store'
 b'\xe5\xa1\x91\xe5\x8f\xb0\xe7\x9f\xb3\xe5\x8c\x96\xe6\x9c\x9d\xe8\x8c\xb6\xe6\x9a\xae\xe9\x85\x92 Bar - MinLife Store']


If you want to leverage ANN techniques to speed up predictions, please check out the [Efficient serving](https://www.tensorflow.org/recommenders/examples/efficient_serving) tutorial.

## Exporting the model

We simply export the `BruteForce` layer we created above:

In [18]:
# Export the query model.
tf.saved_model.save(index, "exported-retrieval/123")

INFO:tensorflow:Assets written to: exported-retrieval/123\assets


INFO:tensorflow:Assets written to: exported-retrieval/123\assets


We will deploy the model with TensorFlow Serving soon.

In [19]:
# Zip the SavedModel folder for easier download
!zip -r exported-retrieval.zip exported-retrieval/ 

'zip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
